In [1]:
import openseespy.opensees as ops
import meshio
import numpy as np

# Leer malla .msh
mesh = meshio.read("Pro_malla.msh")

# Extraer nodos y tetraedros
nodes = mesh.points
tets = [cell.data for cell in mesh.cells if cell.type == "tetra"][0]

# Iniciar modelo 3D
ops.wipe()
ops.model("BasicBuilder", "-ndm", 3, "-ndf", 3)

# Crear nodos
for i, coord in enumerate(nodes):
    ops.node(i + 1, *coord[:3])  # OpenSees indexa desde 1

# Crear material
# Fuente típica PLA: E ≈ 3.5 GPa, ν ≈ 0.36, densidad ≈ 1250 kg/m³
ops.nDMaterial("ElasticIsotropic", 1, 3.5e9, 0.36, 1250)


# Revisar primeros elementos para verificar conectividad
# Crear elementos tetraédricos
for i, conn in enumerate(tets):
    if len(conn) != 4 or len(set(conn)) != 4:
        print(f"❌ Elemento inválido {i}: {conn}")
        continue
    node_ids = [int(nid + 1) for nid in conn]  # 🔧 conversión a int puro
    ops.element("FourNodeTetrahedron", i + 1, *node_ids, 1)
